# Projekt-Dokumentation: Der Ritter der verlorenen Seelen

**Projekttitel:** Der Ritter der verlorenen Seelen
**Plattform:** Java (Swing/AWT)
**Modul:** Programmieren 1 - Prof. Dr. Jan Rexilius
**Entwickler:** Marvin Sandmeier, Johann Issa, Ralf Schmidt, Marc Mittelstedt

Diese Dokumentation hält den inkrementellen Aufbau des Spiels fest und kommentiert die wichtigsten architektonischen und technischen Entscheidungen.

---

## Teil 1: Entwicklung der Spiellogik

Dieser Abschnitt beschreibt die Implementierung der zentralen Spielschleife, der Entitäten und der Physik.

### Schritt 1: Das Grundgerüst, Game Loop und Architektur

Die Entwicklung begann mit der Etablierung einer stabilen und steuerbaren Architektur, die eine saubere Trennung von UI und Logik ermöglicht.

> **Verwendeter Prompt:**
> "Definiere in Java Swing eine Hauptklasse `GamePanel` als `JPanel` mit einem `Timer` für eine konstante Game-Loop (ca. 60 FPS). Nutze die Trennung in `paintComponent()` (Zeichnen) und `actionPerformed()` (Logik-Update) und implementiere ein Zustandssystem (`GameState`) für Menü und Spiel."

* **Designentscheidung (Architektur):** Die Trennung der Hauptkomponenten in `Game` (JFrame Setup), `GamePanel` (Game-Loop und Controller) und `MenuManager` (UI-Logik) folgt dem **MVC-Prinzip** (Model-View-Controller) und sorgt für eine saubere Code-Basis.
* **Technische Notiz (Timing):** Im `GamePanel.java` wird der `javax.swing.Timer` mit einer Verzögerung von 16 ms (ca. 62.5 FPS) verwendet. Die anschließende Nutzung von Delta Time (`dt = timer.getDelay() / 1000.0;`) in der `update`-Methode der `MovingPlatform` stellt sicher, dass die Bewegung **framerate-unabhängig** erfolgt.

---

### Schritt 2: Skalierung und Kollisions-Basis

Um das Spiel zukunftssicher zu machen, wurde frühzeitig ein Skalierungssystem eingeführt.

> **Verwendeter Prompt:**
> "Wie kann ich in Java eine globale Skalierungsvariable definieren, die in allen Klassen, die visuelle Elemente zeichnen oder positionieren, verwendet wird? Definiere einen `Tile` mit einer `getRect()` Methode."

* **Designentscheidung (Skalierung):** Die Klasse `utils.Zoom` mit der statischen Variable `SCALE` wurde erstellt. Dies ist essenziell für die Anpassung an verschiedene Bildschirmauflösungen, da alle grafischen Elemente (z.B. `TILE_SIZE` in `Level.java`) auf dieser Basis berechnet werden.
* **Technische Notiz (Basis-Kollision):** Die Klasse `Tile.java` dient als Basis für alle Level-Elemente und liefert die Hitbox (`getRect()`), was die Kollisionsabfragen im `Player` und `GamePanel` stark vereinfacht (AABB-Kollision).

---

### Schritt 3: Entitäten-Management und Stomp-Logik

Die Spiellogik erfordert das Management von Spielern und Gegnern (`Enemy`/`MovingEnemy`) sowie eine spezielle Kollisionsregel für das Besiegen von Gegnern (Stomp-Mechanik).

> **Verwendeter Prompt:**
> "Implementiere die Gegner-Kollisionslogik im `GamePanel`, wobei der Spieler nur dann Punkte erhält und abprallt (`player.bounceAfterStomp()`), wenn er fällt (`player.isFalling()`) und seine Y-Position über der des Gegners liegt."

* **Designentscheidung (Kollisionspriorität):** Die `handleEnemyCollision` Methode im `GamePanel` priorisiert die vertikale Kollision. Dies ist die zentrale Regel, die dem Spiel seinen "Mario-Style" verleiht:
    ```java
    // Spieler springt von oben auf den Gegner
    if (player.isFalling() && player.getY() < en.getY()) {
        removeEnemy(en); // Gegner stirbt
        // ...
    }
    ```
* **Technische Notiz (Thread-Sicherheit):** Im `GamePanel.actionPerformed` werden Kopien der Entitäten-Listen (`new ArrayList<>(enemies)`) für die Kollisions- und Update-Schleifen verwendet. Dies vermeidet `ConcurrentModificationException`-Fehler, wenn ein Gegner während der Iteration durch die `removeEnemy` Methode stirbt.

---

### Schritt 4: Erweiterte Physik: Bewegliche Plattformen

Die `MovingPlatform` erweitert die Komplexität der Physik, indem sie dem Spieler ermöglicht, sich mit ihr zu bewegen.

> **Verwendeter Prompt:**
> "Schreibe eine Java-Klasse `MovingPlatform`, die sich entlang eines Vektors (Start-X/Y zu End-X/Y) bewegt. Nutze Vektorprojektion, um die Richtung bei Erreichen der Grenzen umzukehren."

* **Designentscheidung (Vektorielle Bewegung):** Die Plattform nutzt Vektormathematik zur Berechnung der Bewegung entlang einer Linie.
    $$
    proj = \frac{(\vec{P} - \vec{A}) \cdot (\vec{B} - \vec{A})}{\|\vec{B} - \vec{A}\|^2}
    $$
    Ist $proj \ge 1$ oder $proj \le 0$, wird die Richtung umgekehrt. Dies gewährleistet eine präzise Bewegung auf diagonalen Strecken.
* **Designentscheidung (UX):** Im `draw`-Loop der `MovingPlatform` wird die Bewegungslinie (`g.drawLine`) gezeichnet, um dem Spieler die Laufbahn vorab zu zeigen.

---

## Teil 2: Aufbau von GUI und Level-System

Dieser Abschnitt dokumentiert die Erstellung der Level-Daten, des Kamerasystems und des Menü-Managements.

### Schritt 5: Level-Design und Datenhaltung

Die Level wurden direkt im Code erstellt, um die präzise Platzierung der Plattformen im Kontext der Skalierung zu gewährleisten.

> **Verwendeter Prompt:**
> "Implementiere eine statische Fabrik-Methode `createSampleLevel(index)` in der Klasse `Level.java`, die `Tile`-Objekte basierend auf einer for-Schleife und Offset-Berechnungen (`TILE_SIZE`, `PLAYER_HEIGHT`) generiert."

* **Designentscheidung (Level-Speicherung):** Die Level werden **hartkodiert** in der Methode `createSampleLevel(index)` definiert. Dies erlaubt die Verwendung von Schleifen (z.B. für Boden oder Rampen) und präzisen, skalierten Offsets, ohne ein komplexes externes Parser-System implementieren zu müssen.
* **Technische Notiz (Parallax-Kamera):** Die `draw`-Methode in `Level.java` nutzt die Kamerakoordinate (`camX`) des Spielers, um den Hintergrund nur mit der halben Geschwindigkeit zu verschieben (`parallaxShift = camX / 2`). Dies erzeugt den klassischen **Parallax-Effekt**, der dem Spiel Tiefe verleiht.

---

### Schritt 6: User Interface (Menüs und Steuerung)

Der `MenuManager` steuert die gesamte Benutzeroberfläche außerhalb des Spielzustands.

> **Verwendeter Prompt:**
> "Erstelle einen `MenuManager`, der auf `GameState` reagiert und dynamische Buttons für Level-Auswahl generiert. Implementiere einen Hover-Effekt für die Buttons."

* **Designentscheidung (Zustandsmaschine):** Der `GamePanel` nutzt die `GameState`-Enum als zentrale Steuergröße.
* **Technische Notiz (Eingabe):** Der `GamePanel` verwendet `InputMap` und `ActionMap` (anstelle eines einfachen `KeyListener`). Dies ist technisch sauberer, da Tastenanschläge korrekt dem fokussierten `JPanel` zugeordnet werden.
* **Designentscheidung (Flüssige UX):** Jeder wichtige Übergang (z.B. Menü $\rightarrow$ Start) ruft `game.showLoadingThen(GameState...)` auf. Dies bindet die Klasse `LoadingScreens` (mit 500 ms Dauer) ein, um eine visuell flüssige Transition zu simulieren.

---

### Schritt 7: Scoring und Persistenz

Die Klasse `Storage` sorgt für die Speicherung der Spielergebnisse.

> **Verwendeter Prompt:**
> "Implementiere eine Klasse `Storage` zum Speichern von Highscores für mehrere Levels (`levelHighscores`) und einer Gesamtpunktzahl, die die besten Einzel-Level-Scores summiert."

* **Designentscheidung (Highscores):** Die Methode `updateTotalScore()` in `Storage.java` stellt sicher, dass die **Gesamtpunktzahl** nur aus den besten jemals in den einzelnen Levels erzielten Scores besteht (Arcade-Prinzip).
* **Technische Notiz (Level-Zählung):** Die dynamische Generierung der Level-Buttons im `MenuManager` verwendet die Konstante `totalNumberOfLevels` aus `Storage.java`, was eine einfache Erweiterung der Level-Anzahl ermöglicht, ohne den `MenuManager` anpassen zu müssen.